In [1]:
from datasets import load_dataset

dataset = load_dataset("allenai/openbookqa", "additional", split=["train", "test"])

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/4957 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/500 [00:00<?, ? examples/s]

In [2]:
NUM_PERMUTE_QUESTION=None
EACH_HAS=24
NUM_CHOICE=4

In [3]:
dataset[0][0]

{'id': '7-980',
 'question_stem': 'The sun is responsible for',
 'choices': {'text': ['puppies learning new tricks',
   'children growing up and getting old',
   'flowers wilting in a vase',
   'plants sprouting, blooming and wilting'],
  'label': ['A', 'B', 'C', 'D']},
 'answerKey': 'D',
 'fact1': 'the sun is the source of energy for physical cycles on Earth',
 'humanScore': 1.0,
 'clarity': 2.0,
 'turkIdAnonymized': 'b356d338b7'}

In [4]:
import numpy as np


def get_prompt(dataset, index, location=-1, has_choice=False):
    json_line = dataset[index]
    question = json_line["question_stem"]
    choices = json_line["choices"]
    choice_texts = choices["text"]
    answer_key = json_line["answerKey"]
    answer_key_idx = ord(answer_key[0]) - ord("A")
    answer_text = choices["text"][answer_key_idx]
    if location>-1:
            perm=np.random.permutation(len(choice_texts)-1)
            choice_texts.remove(answer_text)
            choice_texts = [choice_texts[perm[i]].replace("\n", " ") for i in range(len(choice_texts))]
            choice_texts.insert(location, answer_text)
    else:
        perm=list(range(NUM_CHOICE))
        choice_texts = [choice_texts[perm[i]].replace("\n", " ") for i in range(len(choice_texts))]
    candidates = " ".join(
        [
            f"({label if has_choice else ' '}) {text}"
            for text, label in zip(choice_texts, choices["label"])
        ]
    ).replace("\n", " ")
    
    fact = f"{json_line['fact1']}{'' if json_line['fact1'][-1]=='.' else '.'}"
    prompt = f"Context: {fact} Question: {question} \\n {candidates}"
    return prompt, answer_text

In [5]:
dataset[1][0]

{'id': '8-343',
 'question_stem': 'A person wants to start saving money so that they can afford a nice vacation at the end of the year. After looking over their budget and expenses, they decide the best way to save money is to',
 'choices': {'text': ['make more phone calls',
   'quit eating lunch out',
   'buy less with monopoly money',
   'have lunch with friends'],
  'label': ['A', 'B', 'C', 'D']},
 'answerKey': 'B',
 'fact1': 'using less resources usually causes money to be saved',
 'humanScore': 1.0,
 'clarity': 2.0,
 'turkIdAnonymized': 'b356d338b7'}

In [6]:
from tqdm import tqdm, trange
import pickle
DATASET_NAME='obqa_fact'
if NUM_PERMUTE_QUESTION:
    container=[]
    each_choice_has=EACH_HAS/NUM_CHOICE
    for ques_index in trange(NUM_PERMUTE_QUESTION):
        for idx in range(EACH_HAS):
            location=int(idx/each_choice_has)
            container.append(
                    get_prompt(dataset[1],ques_index,location,  has_choice=False)
                )
    pickle.dump((NUM_PERMUTE_QUESTION, EACH_HAS, container), open(f"{DATASET_NAME}_test_permute.pkl", "wb"))
else:
    for index, dataname in enumerate(['train','test']):
        container=[]
        for ques_index, ques in enumerate(tqdm(dataset[index])):
            container.append(
                    get_prompt(dataset[index], ques_index, has_choice=False)
                )
        pickle.dump(container, open(f"{DATASET_NAME}_{dataname}.pkl", "wb"))

100%|██████████| 500/500 [00:00<00:00, 5266.70it/s]
